In [1]:
import numpy as np
import gradflow.comp_graph as cg

import autograd as ag

In [2]:
d_in = 7
d_out = 1
d_hid = 13

w_A_np = np.random.normal(size=(d_hid, d_in)) # 13 x 7
w_C_np = np.random.normal(size=(d_out, d_hid)) # 1 x 13
x_np = np.random.normal(size=(d_in, 1)) # 7 x 1

x = cg.Value("x", x_np)
w_A = cg.Value("w_A", w_A_np)
w_C = cg.Value("w_C", w_C_np)
linear1 = cg.Dot("linear1")
linear2 = cg.Dot("linear2")

#forward pass
x.forward()
w_A.forward()
w_C.forward()
y1 = linear1.forward(w_A, x)
y = linear2.forward(w_C, linear1)

# the lhs of linear2 is w_C, which has an output shape of w_C
# the rhs of linear2 is w_A x

# the lhs of linear1 is w_A
# the rhs of linear1 is x 


In [3]:
def test_function(input):
    z = ag.numpy.dot(w_A_np, input)
    # z = ag.numpy.tanh(z)
    z = ag.numpy.dot(w_C_np, z)
    # z = ag.numpy.tanh(z)
    return z

In [4]:
atol = 1e-5

y_ag = test_function(x_np)
assert np.allclose(y, y_ag, atol=atol)

In [5]:
# compute backward pass by going in reverse topological order
# linear2.d_out = np.array(1.)
linear2.d_out = np.array([[1.]])
# linear2.d_out = np.array(1.)
print(f"### linear2 backward ###")
_ = linear2.backward()
print(f"### linear1 backward ###")
_ = linear1.backward()

### linear2 backward ###
lhs.d_out.shape: (1, 13)
input.shape, mult.shape, vec.shape = (1, 13), (13, 1), (1, 1)
vjp.shape = (1, 13)
d_lhs.shape: (1, 13)
rhs.d_out.shape: (13, 1)
input.shape, mult.shape, vec.shape = (13, 1), (1, 13), (1, 1)
vjp.shape = (13, 1)
d_rhs.shape: (13, 1)
### linear1 backward ###
lhs.d_out.shape: (13, 7)
input.shape, mult.shape, vec.shape = (13, 7), (7, 1), (13, 1)
vjp.shape = (13, 7)
d_lhs.shape: (13, 7)
rhs.d_out.shape: (7, 1)
input.shape, mult.shape, vec.shape = (7, 1), (13, 7), (13, 1)
vjp.shape = (7, 1)
d_rhs.shape: (7, 1)


In [10]:
print(w_A.out.shape)
print(w_A.d_out.shape)

(13, 7)
(13, 7)


In [11]:
print(w_C.out.shape)
print(w_C.d_out.shape)

(1, 13)
(1, 13)


In [12]:
print(x.out.shape)
print(x.d_out.shape)

(7, 1)
(7, 1)
